<a href="https://colab.research.google.com/github/deniseiras/Forex_RNN/blob/main/Forex_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
# Rede RNN (LSTM) - Moeda

Denis M. A. Eiras
Atualizado: 14/09/2023

O notebook objetiva fazer a predição das moedas EURO e DÓLAR dos últimos 10 dias e comparar com os dados observados. Ainda, fazer a predição futura dos próximos 10 dias.

O dataset é disponibilizado através do uso da API yahoo finance.

Tempo aproximado de execução de xxx minutos

# TODO - O dataframe ... serve só para a predição diária

## Inicialização das bibliotecas e parâmetros gerais


In [ ]:
import numpy as np
from keras.models import Model, Sequential, model_from_json
from keras.layers import LSTM, Input, Dropout, Dense, RepeatVector, TimeDistributed, Conv1D, Flatten, MaxPooling1D, \
  BatchNormalization, Activation
from keras.regularizers import l2
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn import metrics
import seaborn as sns
import tensorflow as tf
from google.colab import drive
import pickle
#import glob
!pip install yfinance
import yfinance as yf
from IPython import display
import sys
import warnings
from time import time
from datetime import datetime, timedelta
from random import uniform
import json

warnings.filterwarnings('ignore')
print("")
print("Python version:")
print (sys.version)
print("\nTensorflow version:")
print(tf.__version__)
print('')

starts_at = datetime.now()
print(f'Starting at {starts_at}')


Informações sobre o dólar USD

In [ ]:
eurobrl = "EURBRL=X"
usdbrl = "USDBRL=X"
btcusd = "BTC-USD"
msft = yf.Ticker(usdbrl)

# get stock info
print(msft.info)

# get historical market data
hist = msft.history(period="max")

# show actions (dividends, splits)
# print(msft.actions)

# show dividends
print(msft.dividends)

# show splits
print(msft.splits)

# show major holders
print(msft.major_holders)

# show institutional holders
print(msft.institutional_holders)


## Leitura e tratamento do Dataset



Requisição de dados para API *Yahoo Finance* e visualização de informações básicas.

---



In [ ]:
arr_acoes = [eurobrl, usdbrl]

data = yf.download(arr_acoes)

df = data["Close"]
df.dropna(axis=0, inplace=True)
print("Data Inicial: ", df.index.min())
print("Data Final  : ", df.index.max())
print("Tamanho     : ", len(df))
print('')

fig, ax = plt.subplots(figsize=(15, 10), nrows=1, ncols=1)
sns.lineplot(data=df, palette='inferno', ax=ax,hue_order=arr_acoes)
ax.set_xlabel('Anos')
ax.set_title('Preço de Encerramento')

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.describe()

Seleção e visualização do dataset de Treino e Teste.

---



In [ ]:
today = datetime.today()

# prev últimos dias úteis
dias_teste = 20

data_corte_treino = today - timedelta(days=1)
while True:
  data_treino = df.loc[(df.index <= data_corte_treino)]
  data_teste = df.loc[(df.index > data_corte_treino)]
  if len(data_teste) == dias_teste:
    break
  data_corte_treino = data_corte_treino - timedelta(days=1)


print("Data Inicial Treino: ", data_treino.index.min())
print("Data Final   Treino: ", data_treino.index.max())
print("Tamanho      Treino: ", len(data_treino))
print('')

print("Data Inicial Teste: ", data_teste.index.min())
print("Data Final   Teste: ", data_teste.index.max())
print("Tamanho      Teste: ", len(data_teste))

fig, ax = plt.subplots(figsize=(12, 8), nrows=1, ncols=1)
sns.lineplot(data=data_treino, palette="inferno", ax=ax, hue_order=arr_acoes)
ax.set_xlabel('Anos')
ax.set_title('Dataset de Treino')
# ax.set_ylim([0, 80])
plt.show()
print('')


# variáveis não usadas para treino, apenas visualização
tmp_data_corte_treino_30dias = today - timedelta(days=30)
tmp_data_treino_30dias = df.loc[(df.index >= tmp_data_corte_treino_30dias)]
fig, ax = plt.subplots(figsize=(12, 8), nrows=1, ncols=1)
sns.lineplot(data=tmp_data_treino_30dias, palette="inferno", ax=ax, hue_order=arr_acoes)
ax.set_xlabel('Dias')
ax.set_title('Dataset de Treino - último mês')
# ax.set_ylim([0, 80])
plt.show()
print('')

fig, ax = plt.subplots(figsize=(12, 8), nrows=1, ncols=1)
sns.lineplot(data=data_teste, palette="inferno", ax=ax, hue_order=arr_acoes)
ax.set_xlabel('Dias')
ax.set_title(f'Dataset de Teste - últimos {dias_teste} dias')
plt.show()

## Pré Processamento

Função utilizada na transformação da base de dados em 3 dimensões, contendo:

1.   Uma sequência ou mais de exemplos (Batch Size);
2.   Um ponto ou mais de observações em série temporal (Time Steps);
3.   Numero de váriaveis contidas em cada Time Step (Features).

---



In [ ]:
def preprocess(dataset, stock, TimeSteps, TesteLen):

    # StandardScaler
    n = len(dataset[[stock]])
    scaler = StandardScaler()
    scaler = scaler.fit(dataset[[stock]])

    df_scal = scaler.transform(dataset[[stock]])
    df_scal = df_scal.reshape(n, 1)

    X = df_scal
    X_samples = list()
    y_samples = list()

    NumerOfRows = len(X)
    for i in range(int(TimeSteps) , NumerOfRows , 1):
        x_sample = X[i-TimeSteps:i]
        y_sample = X[i]
        X_samples.append(x_sample)
        y_samples.append(y_sample)

    X_data=np.array(X_samples)
    X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], 1)

    y_data=np.array(y_samples)
    y_data=y_data.reshape(y_data.shape[0], 1)

    # Separando dataset entre treino e teste
    if TesteLen > 0:
      X_train=X_data[:-TesteLen]
      X_test=X_data[-TesteLen:]
      y_train=y_data[:-TesteLen]
      y_test=y_data[-TesteLen:]
    else:
      X_train=X_data
      X_test=[]
      y_train=y_data
      y_test=[]
    return scaler, X_train, X_test, y_train, y_test


def preprocess_CNN1D(dataset, stock, TesteLen):
    # StandardScaler
    n = len(dataset[[stock]])
    scaler = StandardScaler()
    scaler = scaler.fit(dataset[[stock]])

    df_scal = scaler.transform(dataset[[stock]])
    df_scal = df_scal.reshape(n, 1)

    X = np.array(df_scal)
    # Separando dataset entre treino e teste
    if TesteLen > 0:
      X_train = X[:-TesteLen]
      X_test = X[-TesteLen:]
    else:
      X_train = X_train
      X_test = []

    y_train, y_test = X_train, X_test
    X_train = X_train.reshape(X_train.shape[0], 1, 1)
    X_test = X_test.reshape(X_test.shape[0], 1, 1)

    return scaler, X_train, X_test, y_train, y_test




Exemplo de transformação do dataset, utilizando 10 observações por batch, separando os ultimos len(data_teste) registros para avaliação dos melhores modelos.



In [ ]:
timesteps = 10
print("LSTM data\n\n")
scaler, X_train, X_test, y_train, y_test = preprocess(df, eurobrl, timesteps, len(data_teste))
for inp, out in zip(X_train[0:3], y_train[0:3]):
  print(inp,'--', out)

print("CNN1D data\n\n")
scaler, X_train, X_test, y_train, y_test = preprocess_CNN1D(df, eurobrl, len(data_teste))
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
for inp, out in zip(X_train[0:3], y_train[0:3]):
  print(inp,'--', out)

## Construção dos modelos RNN

Para desenvolvimento do projeto foram testadas dois modelos de RNN, ambas submetidas ao mesmo dataset e a variação em paramêtros.

Modelo v1:
- 1 camada LSTM com ativação relu
- 1 camada Densa
- 1 camada de ativação linear

Modelo v2:
- 1 camada LSTM com ativação relu
- 1 camada Dropout 10%
- 1 camada LSTM com ativação relu e 1/4 do número de neurônios como input
- 1 camada Dropout 10%
- 1 camada Densa
- 1 camada de ativação linear

In [ ]:
def train_model(mod, optimizer, X, y, batch_size, patience, epochs):

  callback = tf.keras.callbacks.EarlyStopping(monitor='val_mean_absolute_error', patience=patience, mode='min',
                                              restore_best_weights=True)

  # only Adam implemented
  if optimizer == "Adam":
    adam = tf.keras.optimizers.Adam()
    mod.compile(loss=tf.losses.MeanSquaredError(),optimizer=adam, metrics=[tf.metrics.MeanAbsoluteError(), tf.metrics.MeanSquaredError()])

  display.display(mod.summary())
  hist = mod.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[callback], verbose=0,
                  shuffle=False)

  loss_metric_train = mod.evaluate(X, y, verbose=0, batch_size=batch_size, use_multiprocessing=True)
  display.display( 'TRAINING set. Loss = {} , MAE = {}'.format(loss_metric_train[0], loss_metric_train[1]))

  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  plt.plot(hist.history['loss'], label="Loss Treino")
  plt.plot(hist.history['val_loss'], label="Loss Validação")
  plt.plot(hist.history['mean_absolute_error'], label='MAE Validação')
  plt.ylim([0.0, 0.2])
  plt.xlim([0.0, epochs])
  plt.legend()
  plt.show()

  return mod, hist


def model_type(mod_type, unit, optimizer, timesteps, X, y, epochs=60, batch_size=32, patience=5):
  func = globals()[mod_type]
  return func(unit, optimizer, timesteps, X, y, epochs, batch_size, patience)


def LSTM_v1(unit, optimizer, timesteps, X, y, epochs, batch_size, patience):

    mod = Sequential()
    mod.add(LSTM(int(unit), activation='relu', input_shape=(timesteps, 1), return_sequences=False))
    mod.add(Dense(1, activation='linear'))

    mod, hist = train_model(mod, optimizer, X, y, batch_size, patience, epochs)

    return mod, hist


def LSTM_v2(unit, optimizer, timesteps, X, y, epochs, batch_size, patience, l2_reg=0.001):

    mod = Sequential()
    mod.add(LSTM(int(unit), activation='relu', input_shape=(timesteps, 1), kernel_regularizer=l2(l2_reg),
                 return_sequences=False))
    mod.add(Dense(1, activation='linear'))

    mod, hist = train_model(mod, optimizer, X, y, batch_size, patience, epochs)

    return mod, hist


def LSTM_v3(unit, optimizer, timesteps, X, y, epochs, batch_size, patience, dropout_rate=0.3, l2_reg=0.001):

    mod = Sequential()
    mod.add(LSTM(int(unit), activation='relu', input_shape=(timesteps, 1), kernel_regularizer=l2(l2_reg),
                 return_sequences=False))
    mod.add(Dropout(dropout_rate))
    mod.add(Dense(1, activation='linear'))

    mod, hist = train_model(mod, optimizer, X, y, batch_size, patience, epochs)

    return mod, hist


def LSTM_v4(unit, optimizer, timesteps, X, y, epochs, batch_size, patience, dropout_rate=0.3):

    mod = Sequential()
    mod.add(LSTM(int(unit), activation='relu', input_shape=(timesteps, 1), return_sequences=True))
    mod.add(Dropout(0.1))
    mod.add(LSTM(int(unit/4), activation='relu', return_sequences=False ))
    mod.add(Dropout(0.1))
    mod.add(Dense(1, activation='linear'))

    mod, hist = train_model(mod, optimizer, X, y, batch_size, patience, epochs)

    return mod, hist


def CNN_v1(unit, optimizer, timesteps, X, y, epochs, batch_size, patience):

  mod = Sequential()
  mod.add(Conv1D(filters=unit, kernel_size=2, activation='relu', input_shape=(1,1)))
  # mod.add(BatchNormalization())
  mod.add(Activation("relu"))
  # mod.add(MaxPooling1D(pool_size=3))
  mod.add(Flatten())
  mod.add(Dense(1, activation='linear'))

  mod, hist = train_model(mod, optimizer, X, y, batch_size, patience, epochs)
  return mod, hist

## Treinamento e Hiperparâmetrização

Parâmetros variados

1. Modelo: v1 ou v2
2. Numéro de Neurônios:  [64, 128]
3. Batch size: [16, 32, 64]


In [ ]:
all_results = pd.DataFrame(columns=["model_type","stock","unit","optimizer", "batch_size", "loss","mse", "timesteps"])

arr_acoes_hyperparams = arr_acoes
model_hyperparams = ['LSTM_v1', 'LSTM_v2', 'LSTM_v3']
unit_hyperparams = [32, 64, 128]
batch_hyperparams = [8, 16, 32]
arr_timesteps_hyperparams = [10, 20]

TotalFeatures = 1
epochs = 100
times_exec = 3


# for testing
# arr_acoes_hyperparams = arr_acoes
# model_hyperparams = ['LSTM_v1', 'LSTM_v2']
# unit_hyperparams = [2, 4]
# batch_hyperparams = [8, 16]
# arr_timesteps_hyperparams = [5, 10]
# TotalFeatures = 1
# epochs = 3
# times_exec = 2

best_models_df = pd.DataFrame(columns=["model_type", "model", "stock","unit","optimizer", "batch_size", "mse", "timesteps"])

for times in range(times_exec):
  for stock in arr_acoes_hyperparams:
    for timesteps in arr_timesteps_hyperparams:
      for model_type_str in model_hyperparams:
        if model_type_str[:4] == 'LSTM':
          scaler, X_train, X_test, y_train, y_test = preprocess(df, stock, timesteps, len(data_teste))
        elif model_type_str[:3] == 'CNN':
          scaler, X_train, X_test, y_train, y_test = preprocess_CNN1D(df, stock, len(data_teste))
        for optimizer in ['Adam']:
          for unit in unit_hyperparams:
            for batch_size in batch_hyperparams:
              inittime = time()
              print(f"Treinando {stock} com {model_type_str} , units={unit}, batch={batch_size}, timesteps={timesteps}")
              m, h = model_type(model_type_str, unit=unit, optimizer=optimizer, timesteps=timesteps, X=X_train, y=y_train, epochs=epochs, batch_size=batch_size)
              endtime = time()
              print(f'Tempo de treino: {(endtime-inittime)/60} minutos' )
              y_pred = m.predict(X_test)
              score = m.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
              mod_cfg_res = {'model_type': model_type_str, 'stock':stock,'unit':unit, 'optimizer':optimizer,
                                            "loss":score[0], "mae":score[1], "mse":score[2], "batch_size":batch_size, "timesteps":timesteps}
              all_results = all_results.append(mod_cfg_res, ignore_index=True)

              mod_cfg_dic = {'model_type': model_type_str, 'stock':stock,'unit':unit, 'optimizer':optimizer,
                             "batch_size":batch_size, "timesteps":timesteps}

              # save the model architecture to a JSON string
              fprefix = f"{stock}_{model_type_str}_{unit}_{batch_size}_{timesteps}"
              with open(f"{fprefix}.json", "w") as json_file:
                model_json = m.to_json()
                json_file.write(model_json)
                # serialize weights to HDF5
                m.save_weights(f"{fprefix}.h5")

              del m, h

## Avaliação na base de testes
Desempenho ordenado por moeda e menor mse na base de testes

In [ ]:
all_results.sort_values(by=['stock','mse'])

É considerada a melhor média, em termos de MAE, das três execuções

In [ ]:
# MElhor médias das execuções
print(len(all_results))
mean_mse = all_results.groupby(['stock', 'model_type', 'unit', 'batch_size', 'optimizer', 'timesteps'] )['mse'].mean()
mean_mse = mean_mse.reset_index()
print(len(mean_mse))
display.display(mean_mse.sort_values(by=['mse']))



Seleção dos melhores modelos de cada arquitetura

In [ ]:

best_models_arr = []

for model_str in model_hyperparams:
  for stock in arr_acoes:
    best_model = mean_mse[mean_mse['stock'] == stock].sort_values('mse', ascending=True).head(1)
    # best_model = mean_mse[mean_mse['stock'] == stock].sort_values(by=['mse'])
    best_model_obj=best_model.iloc[0]

    fprefix = f"{stock}_{best_model_obj['model_type']}_{best_model_obj['unit']}_{best_model_obj['batch_size']}_{best_model_obj['timesteps']}"
    # load json and create model
    json_file = open(f'{fprefix}.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(f"{fprefix}.h5")
    best_model['model'] = loaded_model

    best_models_arr.append(pd.DataFrame(best_model))

best_models_df = pd.concat(best_models_arr)
del best_models_arr
print("Melhores modelos")
display.display(best_models_df.sort_values('mse', ascending=True))
# display.display(best_models_df)

### Boxplot

Construção dos dados de predição, também usados no Boxplot e Swarmplot

---

In [ ]:
aval = pd.DataFrame()

for index, setup in best_models_df.iterrows():

  scaler, X_train, X_test, y_train, y_test = preprocess(df, setup["stock"], setup["timesteps"] , len(data_teste))

  mod = setup['model']

  y_pred = mod.predict(X_test)
  y_pred = scaler.inverse_transform(y_pred)
  y_real = scaler.inverse_transform(y_test)

  aval[f'{setup["stock"]}'] = y_pred[:,0]
  aval[f'{setup["stock"]}_Real'] = y_real[:,0]



Figura com boxplot e swarmplot para os melhores modelos

---

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7), nrows=1, ncols=2)
sns.boxplot(data=aval.filter(regex=usdbrl), dodge=False, palette="Blues", ax=ax[0])
sns.swarmplot(data=aval.filter(regex=usdbrl), color=".25", alpha=0.8, ax=ax[0])

sns.boxplot(data=aval.filter(regex=eurobrl), dodge=False, palette="Blues", ax=ax[1])
sns.swarmplot(data=aval.filter(regex=eurobrl), color=".25", alpha=0.8, ax=ax[1])

ax[1].set_title("Performance Boxplot")

### Performance

Figura predição por moeda

In [ ]:
def plot_aval(aval):
  fig, ax = plt.subplots(figsize=(30, 12), nrows=1, ncols=2)

  sns.lineplot(data=aval.filter(regex=usdbrl), palette="inferno", ax=ax[0], hue_order=[f'{usdbrl}_Real' ,usdbrl])
  ax[0].set_xlabel('Dias')
  ax[0].set_title(usdbrl)

  sns.lineplot(data=aval.filter(regex=eurobrl), palette="inferno", ax=ax[1], hue_order=[f'{eurobrl}_Real', eurobrl])
  ax[1].set_xlabel('Dias')
  ax[1].set_title(eurobrl)

plot_aval(aval)
print(aval)

## Avaliação no futuro


In [ ]:
aval_fut = pd.DataFrame()

sharpness = None
test_fut_days = 10
for index, setup in best_models_df.iterrows():

  scaler, X_train, X_test, y_train, y_test = preprocess(df, setup['stock'], setup["timesteps"], test_fut_days)
  inittime = time()
  print(f"Usando {setup['stock']} com {setup['model_type']} , units={setup['unit']}, batch={setup['batch_size']}, \
    timesteps={setup['timesteps']}")

  mod, _ = model_type(setup['model_type'], setup['unit'], setup['optimizer'], setup['timesteps'], X_train, y_train,
                      epochs=epochs, batch_size=setup['batch_size'])

  m = setup['model']
  endtime = time()
  print(f'Tempo de treino: {(endtime-inittime)/60} minutos' )

  # Initial sequence for prediction
  initial_sequence = X_test[0]
  # Predict future values iteratively
  predicted_values = []

  for _ in range(test_fut_days):
    # Predict the next step
    next_step = m.predict(np.array([initial_sequence]))
    if sharpness is not None:
      next_step[0, 0] = next_step[0, 0] + next_step[0, 0]*uniform(-sharpness, sharpness)
    # Append the predicted value
    predicted_values.append(next_step[0, 0])
    # Update the initial sequence by removing the oldest value and appending the predicted value
    initial_sequence = np.roll(initial_sequence, shift=-1, axis=0)
    initial_sequence[-1] = next_step[0, 0]

  y_pred = scaler.inverse_transform(np.array(predicted_values).reshape(-1, 1))
  y_real = scaler.inverse_transform(y_test)
  aval_fut[f'{setup["stock"]}'] = y_pred[:,0]
  aval_fut[f'{setup["stock"]}_Real'] = y_real[:,0]



### Boxplot

Figura com boxplot e swarmplot para os melhores modelos

---

In [ ]:
fig, ax = plt.subplots(figsize=(20, 7), nrows=1, ncols=2)
sns.boxplot(data=aval_fut.filter(regex=usdbrl), dodge=False, palette="Blues", ax=ax[0])
sns.swarmplot(data=aval_fut.filter(regex=usdbrl), color=".25", alpha=0.8, ax=ax[0])

sns.boxplot(data=aval_fut.filter(regex=eurobrl), dodge=False, palette="Blues", ax=ax[1])
sns.swarmplot(data=aval_fut.filter(regex=eurobrl), color=".25", alpha=0.8, ax=ax[1])

ax[1].set_title("Performance Boxplot")

### Performance

Figura predição por moeda

In [ ]:
plot_aval(aval_fut)
print(aval_fut)
aval_err = pd.DataFrame(columns=['EURBRL_bias', 'USDBRL_bias'])
aval_err['EURBRL_bias'] = 100*abs(aval_fut['EURBRL=X']-aval_fut['EURBRL=X_Real'])/aval_fut['EURBRL=X_Real']
aval_err['USDBRL_bias'] = 100*abs(aval_fut['USDBRL=X']-aval_fut['USDBRL=X_Real'])/aval_fut['USDBRL=X_Real']
print(aval_err)

In [ ]:
ends_at = datetime.now()
print(f'End of execution at {ends_at}')
print(f'Total time of execution = {ends_at-starts_at}')

